# ECE1508 Project - AlexNet

In [ ]:
import os
import copy
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import torchvision.datasets as dsets
from torchvision import models, transforms
from sklearn.metrics import precision_recall_fscore_support

## Data Import

I was using Google Colab.

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install kaggle

In [ ]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

In [ ]:
!unzip face-expression-recognition-dataset.zip

## Data Processing

In [ ]:
#device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#import data
data_dir = "/content/images"

#transformations - AlexNet expects 224x224 RGB images
transformations = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),  #convert B&W to 3-channel
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]),
    'validation': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
}

#load data
data_sets = {
    'train': dsets.ImageFolder(os.path.join(data_dir, 'train'), transform=transformations['train']),
    'validation': dsets.ImageFolder(os.path.join(data_dir, 'validation'), transform=transformations['validation'])
}

#splitting training set to make a test set
full_validation = data_sets['validation']
val_size = int(0.5 * len(full_validation))
test_size = len(full_validation) - val_size
validation_dataset, test_dataset = random_split(full_validation, [val_size, test_size], generator=torch.Generator().manual_seed(42))

#create dataloaders
dataloaders = {
    'train': DataLoader(data_sets['train'], batch_size=64, shuffle=True, num_workers=2),
    'validation': DataLoader(validation_dataset, batch_size=64, shuffle=True, num_workers=2),
    'test': DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)
}

## Model

In [ ]:
#pre-trained AlexNet
weights = models.AlexNet_Weights.IMAGENET1K_V1
model = models.alexnet(weights=weights)

#change to 7 classes
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 7)

model = model.to(device)

#optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

## Training

In [ ]:
num_epochs = 50
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    #train and validation each epoch
    for phase in ['train', 'validation']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            #forward pass
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                #backward pass
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            #calcs
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        #copy if it's the best model
        if phase == 'validation' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    scheduler.step()

    print()

#load best model
model.load_state_dict(best_model_wts)
print(f'Best val Acc: {best_acc:4f}')

## Testing

In [ ]:
model.eval()
test_loss = 0.0
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        #forward pass
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        #update loss
        test_loss += loss.item() * inputs.size(0)

        #store predictions and true labels
        all_preds.extend(preds.view(-1).cpu().numpy())
        all_labels.extend(labels.view(-1).cpu().numpy())

avg_loss = test_loss / len(dataloaders['test'].dataset)
accuracy = (np.array(all_preds) == np.array(all_labels)).mean()

#calculations
precision, recall, f1_score, _ = precision_recall_fscore_support(
    all_labels, all_preds, average='weighted', zero_division=0
)

print(f'Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}, '
      f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}')

## Feature Maps

In [ ]:
def load_image(image_path):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  #add batch dimension
    return image

class AlexNet_Modified(nn.Module):
    def __init__(self, original_model):
        super(AlexNet_Modified, self).__init__()
        self.features1 = nn.Sequential(*list(original_model.features.children())[:3])  #up to first conv layer
        self.features2 = nn.Sequential(*list(original_model.features.children())[3:6])  #up to second conv layer

    def forward(self, x):
        x1 = self.features1(x)
        x2 = self.features2(x1)
        return x1, x2

original_model = models.alexnet(pretrained=True)
modified_model = AlexNet_Modified(model)
modified_model = modified_model.to(device)

image_path = '/content/images/train/angry/0.jpg'
image = load_image(image_path).to(device)

#ensure model is in evaluation mode
modified_model.eval()

#forward pass to get feature maps
with torch.no_grad():
    feature_maps1, feature_maps2 = modified_model(image)

def visualize_feature_maps(feature_maps):
    feature_maps = feature_maps.to('cpu').squeeze(0)  #remove batch dimension and move to CPU
    fig, axes = plt.subplots(nrows=1, ncols=min(5, feature_maps.size(0)), figsize=(15, 10))
    for i, ax in enumerate(axes.flat):
        ax.imshow(feature_maps[i].detach(), cmap='gray')
        ax.axis('off')
    plt.show()

#original image
img = Image.open(image_path)
plt.imshow(img, cmap='gray')
plt.title('Original Image')
plt.axis('off')
plt.show()

#first feature map
print("Feature Maps after 1st Conv Layer")
visualize_feature_maps(feature_maps1)

#second feature map
print("Feature Maps after 2nd Conv Layer")
visualize_feature_maps(feature_maps2)

